In [19]:
import PyPDF2
import csv
import pandas as pd
import re
import sys
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
#URL for the webpage
page = urlopen('https://www.ffiec.gov/nicpubweb/content/BHCPRRPT/BHCPR_Peer.htm#2015').read()
soup = BeautifulSoup(page,'lxml')
soup.prettify()
links_list = []
filter_list = []
#Getting all the hyperlinks in the page
for anchor in soup.findAll('a', href=True):
    links_list.append(anchor['href'])
#Filtering all the webpages into a filter_list
filter_list = [data for data in links_list if 'PeerGroup_1' in data]
#Getting the file name
file_name = []
for name in filter_list:
    n1,n2,n3,n4 = name.split("/")
    file_name.append(n4)
#Appending missing strings in each hyperlinks fetched
string_to_append = 'https://www.ffiec.gov/nicpubweb/content/BHCPRRPT/'
filter_list = [string_to_append + filterData for filterData in filter_list]
#Downloading each file in an incremental format
counter=0
for url in filter_list:
    response = requests.get(url)
    with open("C:/Users/Shashwat/Desktop/NEU/Semester 3/Advances in Data Science/Assignment#1/PDF's/%s"%(file_name[counter]), 'wb') as file:
        counter+=1
        file.write(response.content)
#Function to convert pdf Pages to text to dataframes
def Convert(pageNum):
    pageObj = pdfReader.getPage(pageNum)          #'9' is the page number
    pageContent = pageObj.extractText().splitlines()
    myFile = open('Test.txt','w')
    for string in pageContent:
        myFile.write(string+'\n')
    myFile.close()
    #Removing the first line from every Pages
    with open('C:\\Users\\Shashwat\\Documents\\Test.txt', 'r') as fin:
        data = fin.read().splitlines(True)
    with open('C:\\Users\\Shashwat\\Documents\\Test.txt', 'w') as fout:
        fout.writelines(data[1:])
        fout.close()
    
    txt_file = open("C:\\Users\\Shashwat\\Documents\\Test.txt",'r')
    txt_file2 = open("C:\\Users\\Shashwat\\Documents\\Test_FirstDerived.txt",'w')

    #Function to add new Line in between text
    def add_NewLine(var,nvar):
        index = re.search(r'[*0-9]\s*[a-zA-Z]',var.rstrip())
        x = var.index(index.group(0))
        newVar = nvar
        var1 = var[:x+1]
        var2 = var[x+1:]
        newVar = newVar + '\n' + var1
        if((re.search(r'[*0-9]\s*[a-zA-Z]',var2.rstrip())!=None)):
            return(add_NewLine(var2,newVar))
        newVar = newVar+ '\n' + var2   
        return newVar
    #Function to add new Line when colon is there
    def colon_NewLine(varb):
        index2 = re.search(r'[*:]\s*[a-zA-Z]',varb.rstrip())
        loc = varb.index(index2.group(0))
        varb = varb[:loc+1]+'\n'+varb[loc+1:]
        return varb
    #Loop to check and call above 2 functions
    for li in txt_file:
        index = re.search(r'[*0-9]\s*[a-zA-Z]',li.rstrip())    
        if(index != None):
            pos = li.index(index.group(0))
            if (pos>50):            
                ret = add_NewLine(li,"")
                if((re.search(r'[*:]\s*[a-zA-Z]',ret.rstrip())!=None)):
                    updRet = colon_NewLine(ret)
                    txt_file2.writelines(updRet)
                else:
                    txt_file2.writelines(ret)
            else:
                txt_file2.writelines(li)
        else:        
            txt_file2.writelines(li)
    txt_file2.close()
    #Removing WhiteSpaces with Comma's
    pattern = re.compile(r'(\s){5,}')
    txt_file3 = open("C:\\Users\\Shashwat\\Documents\\Test_FirstDerived.txt",'r')
    txt_file4 = open("C:\\Users\\Shashwat\\Documents\\Test_Final.txt",'w')
    for lines in txt_file3:
        lines = re.sub(',','-',lines)
        lines = re.sub(pattern, ', ', lines)
        txt_file4.writelines(lines)
    txt_file3.close()
    txt_file4.close()
    #Writing into a csv file
    txt_Final = open("C:\\Users\\Shashwat\\Documents\\Test_Final.txt",'r')
    df = pd.DataFrame([sub.split(",") for sub in txt_Final])
    return df

#Our Code Begins from here
for name in file_name:
    #import filename
    frames = []
    pdfFileObj = open("C:\\Users\\Shashwat\\Desktop\\NEU\\Semester 3\\Advances in Data Science\\Assignment#1\\PDF's\\%s"%(name),'rb')     #'rb' for read binary mode
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    pdfLength = pdfReader.numPages
    for i in range(1,pdfLength):
        frames.append(Convert(i))
    df_Final = pd.concat(frames)
    df_Final.to_csv("C:\\Users\\Shashwat\\Desktop\\NEU\\Semester 3\\Advances in Data Science\\Assignment#1\\CSVs\\%s.csv"%(name), sep=',')